In [1]:
import pandas as pd
import numpy as np
from utils import *

import warnings
warnings.filterwarnings('ignore')

In [17]:
"""
I have no idea how to include the ingredient and food category information.
So my solution is to aggregate the nutrition information and concatenate the food description.
The ingredients and food categories are not considered in this process.

Also, diet habits are important attributes, for now, I just use one-hot encoding.
"""

df_food = pd.read_csv('../processed_data/user_food.csv', index_col='SEQN')
food_table = df_food.groupby('SEQN')['food_desc'].agg(';'.join).reset_index().set_index('SEQN')

df_demo = pd.read_csv('../processed_data/main_table.csv', index_col='SEQN')
df_demo['label'] = df_demo['label'].astype(int)
df_demo = df_demo.drop(['weight_interview', 'weight_mec', 'years'], axis=1)

df_diet = pd.read_csv('../processed_data/diet_table.csv', index_col='SEQN')

In [18]:
# Merge the data together, and make sure the demographic table is the main table.
df = df_demo.merge(food_table, left_index=True, right_index=True, how='left')
df = df.merge(df_diet, left_index=True, right_index=True, how='left')

In [24]:
# For now, we don't care about recovered users.
df = df.loc[df['label'] != 2]

In [26]:
df.to_csv('../processed_data/full_set.csv')

In [27]:
"""
The class is highly imbalanced, around 40:1.
We also create a balanced setting.
"""
df_sample = df[df['label'] == 1]
df_sample = pd.concat([df_sample, df.sample(3000)]).drop_duplicates()
df_sample.to_csv('../processed_data/sample_set.csv')